# Text Generation with RNNs


In [0]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

print(torch.__version__)


1.3.1


### 1 Dataset
Define the path of the file, you want to read and train the model on


In [0]:
#Dataset used: Bible in German

!wget "https://bible4u.net/static/bible_files/txt/ELB71_txt.tar.gz"

!gunzip ELB71_txt.tar.gz
!tar -xf ELB71_txt.tar

!ls

--2020-01-24 20:25:11--  https://bible4u.net/static/bible_files/txt/ELB71_txt.tar.gz
Resolving bible4u.net (bible4u.net)... 174.138.121.210, 2400:6180:100:d0::d:4001
Connecting to bible4u.net (bible4u.net)|174.138.121.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1259329 (1.2M) [application/x-tar]
Saving to: ‘ELB71_txt.tar.gz’

ELB71_txt.tar.gz    100%[===================>]   1.20M  1.23MB/s    in 1.0s    

2020-01-24 20:25:13 (1.23 MB/s) - ‘ELB71_txt.tar.gz’ saved [1259329/1259329]

ELB71.txt  ELB71_txt.tar  sample_data


In [0]:
'''TODO: set the path of the file'''
path_to_file = './ELB71.txt'
text = open(path_to_file, encoding='utf-8').read()


#### Inspect the dataset
Take a look at the first 250 characters in text

In [0]:
print(text[2000:2500])

g von der Nacht zu scheiden, und sie seien zu Zeichen und zur Bestimmung von Zeiten und Tagen und Jahren;
Gen.1:15 und sie seien zu Lichtern an der Ausdehnung des Himmels, um auf die Erde zu leuchten! Und es ward also.
Gen.1:16 Und Gott machte die zwei großen Lichter: das große Licht zur Beherrschung des Tages, und das kleine Licht zur Beherrschung der Nacht, und die Sterne.
Gen.1:17 Und Gott setzte sie an die Ausdehnung des Himmels, um auf die Erde zu leuchten,
Gen.1:18 und um zu herrschen am T


In [0]:
# The unique characters in the file
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))


85 unique characters


### 2 Process the dataset for the learning task
The task that we want our model to achieve is: given a character, or a sequence of characters, what is the most probable next character?

To achieve this, we will input a sequence of characters to the model, and train the model to predict the output, that is, the following character at each time step. RNNs maintain an internal state that depends on previously seen elements, so information about all characters seen up until a given moment will be taken into account in generating the prediction.

#### Vectorize the text
Before we begin training our RNN model, we'll need to create a numerical representation of our text-based dataset. To do this, we'll generate two lookup tables: one that maps characters to numbers, and a second that maps numbers back to characters. Recall that we just identified the unique characters present in the text.

In [0]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
text_as_int = np.array([char2idx[c] for c in text])

# Create a mapping from indices to characters
idx2char = np.array(vocab)


This gives us an integer representation for each character. Observe that the unique characters (i.e., our vocabulary) in the text are mapped as indices from 0 to len(unique). Let's take a peek at this numerical representation of our dataset:

In [0]:

print('{')
for char,_ in zip(char2idx, range(12)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '"' :   3,
  "'" :   4,
  '(' :   5,
  ')' :   6,
  ',' :   7,
  '-' :   8,
  '.' :   9,
  '/' :  10,
  '0' :  11,
  ...
}




We can also look at how the first part of the text is mapped to an integer representation:

In [0]:
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))



'Elberfelder 1' ---- characters mapped to int ---- > [29 61 51 54 67 55 54 61 53 54 67  1 12]


#### Defining a method to encode one hot labels

In [0]:
def one_hot_encode(arr, n_labels):
    # Initialize the the encoded array
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)

    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.

    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))

    return one_hot


#### Defining a method to make mini-batches for training

In [0]:
def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.

       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''

    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr) // batch_size_total

    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))

    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n + seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n + seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y




## 3 The Recurrent Neural Network (RNN) model


###### Check if GPU is available

In [0]:
train_on_gpu = torch.cuda.is_available()
print ('Training on GPU' if train_on_gpu else 'Training on CPU')

Training on GPU



### Declaring the model

In [0]:
class VanillaCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        '''TODO: define the layers you need for the model'''
        #self.input_layer = nn.Linear(n_hidden + len(vocab), n_hidden)
        self.rnn = nn.RNN(len(vocab), n_hidden, n_layers, batch_first=True)
        #self.hidden_layer = nn.Linear(n_hidden, len(vocab))
        #self.softmax = nn.LogSoftmax(dim=1)
        
        self.fc = nn.Linear(n_hidden, len(vocab))

    def forward(self, x, hidden):
        '''TODO: Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''

        
        #x_cat = torch.cat((x, hidden), 1)
        #hidden_t = self.hidden_layer(x_cat)
        #out = self.softmax(x_cat)
        
        #for h in hidden:
            #print(h.shape)
        
        #for h in hidden:
         #   hidden_t = torch.stack(hidden, dim=0)
        
        #hidden_t = torch.stack(hidden) 
        #print(hidden.shape)
        #print(x.shape)
        #out, hidden_t = self.rnn(x, hidden)
        
        #out = self.fc(out[:, -1, :])
        #print(out)
        
        hidden_t = torch.stack(hidden) # convert tensor tuple to hidden tensor
        
        if (train_on_gpu):
            hidden_t = hidden_t.cuda()
            x = x.cuda()
        
        out, hidden_t = self.rnn(x, hidden_t)
        out = self.fc(out)
        
        
        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        
        hidden = torch.zeros(self.n_layers, batch_size, self.n_hidden)

        return hidden
"""    
class VanillaCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.vocab = vocab
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.rnn = nn.RNN(input_size=len(vocab), hidden_size=n_hidden,
                          num_layers=n_layers, batch_first=False)

    def forward(self, x, hidden):
        '''x is the input and hidden is the hidden/cell state .'''
        # return the final output and the hidden state
        # hidden = torch.cat([h.unsqueeze(0) for h in hidden],0).cuda()
        outputs = []
        for i, h in enumerate(hidden):
            inp = x[:, i, :]
            print(inp.shape)
            print(h.shape)
            out, h = self.rnn(inp, h)
            outputs.append(out)
        return outputs, h

    def init_hidden(self, batch_size, seq_length):
        ''' Initializes hidden state '''
        return torch.zeros(seq_length, self.n_layers, batch_size, 
                           self.n_hidden)"""
    
    

"    \nclass VanillaCharRNN(nn.Module):\n    def __init__(self, vocab, n_hidden=256, n_layers=2,\n                 drop_prob=0.5, lr=0.001):\n        super().__init__()\n        self.vocab = vocab\n        self.drop_prob = drop_prob\n        self.n_layers = n_layers\n        self.n_hidden = n_hidden\n        self.lr = lr\n        self.rnn = nn.RNN(input_size=len(vocab), hidden_size=n_hidden,\n                          num_layers=n_layers, batch_first=False)\n\n    def forward(self, x, hidden):\n        '''x is the input and hidden is the hidden/cell state .'''\n        # return the final output and the hidden state\n        # hidden = torch.cat([h.unsqueeze(0) for h in hidden],0).cuda()\n        outputs = []\n        for i, h in enumerate(hidden):\n            inp = x[:, i, :]\n            print(inp.shape)\n            print(h.shape)\n            out, h = self.rnn(inp, h)\n            outputs.append(out)\n        return outputs, h\n\n    def init_hidden(self, batch_size, seq_length):\n

In [0]:
class LSTMCharRNN(nn.Module):
    def __init__(self, vocab, n_hidden=256, n_layers=2,
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        self.vocab = vocab
        
        '''TODO: define the layers you need for the model'''
        
        self.lstm = nn.LSTM(len(vocab), n_hidden, n_layers, batch_first=True)
        self.fc = nn.Linear(n_hidden, len(vocab))


    def forward(self, x, hidden):
        '''TODO: Forward pass through the network
        x is the input and `hidden` is the hidden/cell state .'''

        hidden_t = torch.stack(hidden) # convert tensor tuple to hidden tensor
        
        if (train_on_gpu):
            hidden_t = hidden_t.cuda()
            x = x.cuda()
        
        out, hidden_t = self.lstm(x, hidden)
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden_t

    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        
        hidden = torch.zeros(2, self.n_layers, batch_size, self.n_hidden)

        return hidden
    
    
    


#### Declaring the train method

In [0]:
def train(model, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network

        Arguments
        ---------

        model: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss

    '''
    model.train()

    opt = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()

    # create training and validation data
    val_idx = int(len(data) * (1 - val_frac))
    data, val_data = data[:val_idx], data[val_idx:]

    if (train_on_gpu):
        model.cuda()

    counter = 0
    n_vocab = len(model.vocab)
    for e in range(epochs):
        # initialize hidden state
        h = model.init_hidden(batch_size)
        #print(h.shape)
        
        '''TODO: use the get_batches function to generate sequences of the desired size'''
        dataset = get_batches(data, batch_size, seq_length)

        for x, y in dataset:
            counter += 1

            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_vocab)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)

            if (train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])
            
            #h_state = torch.tensor([h[0][0]], [h[1]])
            #print(x.shape)
            
            #if (train_on_gpu):
            #    h = (h.cuda())

            # zero accumulated gradients
            model.zero_grad()
            
            '''TODO: feed the current input into the model and generate output'''
            output, h = model(inputs, h)
            
            #print(targets.shape)
            #print(output.shape)
            
            flat_targets = targets.flatten()
            flat_output = output.view(-1, len(vocab))
            
            #print(flat_targets.shape)
            #print(flat_output.shape)

            '''TODO: compute the loss!'''
            loss = criterion(flat_output, flat_targets)
            
            # perform backprop
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(model.parameters(), clip)
            opt.step()

            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = model.init_hidden(batch_size)
                val_losses = []
                model.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_vocab)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)

                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])

                    inputs, targets = x, y
                    if (train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    '''TODO: feed the current input into the model and generate output'''
                    output, val_h = model(inputs, val_h)
                    
                    flat_targets_val = targets.flatten()
                    flat_output_val = output.view(-1, len(vocab))
                    
                    '''TODO: compute the validation loss!'''
                    val_loss = criterion(flat_output_val, flat_targets_val)

                    val_losses.append(val_loss.item())

                print("Epoch: {}/{}...".format(e + 1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))
                
                '''TODO: sample from the model to generate texts'''
                input_eval = 'E'
                print(sample(model, 1000, prime=input_eval, top_k=10))
                
                model.train()  # reset to train mode after iterationg through validation data


##### Defining a method to generate the next character

In [0]:
def predict(model, char, h=None, top_k=None):
    ''' Given a character, predict the next character.
        Returns the predicted character and the hidden state.
    '''
    #print('char: ' + str(char))
    #print('char_idx: ' + str(char2idx[char]))
    # tensor inputs
    
    x = np.array([[char2idx[char]]])
    x = one_hot_encode(x, len(model.vocab))
    inputs = torch.from_numpy(x)

    if (train_on_gpu):
        inputs = inputs.cuda()

    # detach hidden state from history
    h = tuple([each.data for each in h])
    '''TODO: feed the current input into the model and generate output'''
    output, h = model(inputs, h) # TODO

    # get the character probabilities
    p = F.softmax(output, dim=-1).data
    if (train_on_gpu):
        p = p.cpu()  # move to cpu

    # get top characters
    if top_k is None:
        top_ch = np.arange(len(model.vocab))
    else:
        p, top_ch = p.topk(top_k)
        top_ch = top_ch.numpy().squeeze()

    # select the likely next character with some element of randomness
    p = p.numpy().squeeze()
    char = np.random.choice(top_ch, p=p / p.sum())

    # return the encoded value of the predicted char and the hidden state
    return idx2char[char], h



#### Declaring a method to generate new text

In [0]:
def sample(model, size, prime='The', top_k=None):
    if (train_on_gpu):
        model.cuda()
    else:
        model.cpu()

    model.eval()  # eval mode

    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = model.init_hidden(1)
    for ch in prime:
        char, h = predict(model, ch, h, top_k=top_k)

    chars.append(char)

    for ii in range(size):
      '''TODO: pass in the previous character and get a new one'''
      char, h = predict(model, chars[-1], h, top_k=top_k)
      chars.append(char)

    model.train()
    return ''.join(chars)


#### Generate new Text using the RNN model

###### Define and print the net

In [0]:
''''TODO: Try changing the number of units in the network to see how it affects performance'''
n_hidden = 256
n_layers = 2

vanilla_model = VanillaCharRNN(vocab, n_hidden, n_layers)
print(vanilla_model)
lstm_model = LSTMCharRNN(vocab, n_hidden, n_layers)
print(lstm_model)

VanillaCharRNN(
  (rnn): RNN(85, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=85, bias=True)
)
LSTMCharRNN(
  (lstm): LSTM(85, 256, num_layers=2, batch_first=True)
  (fc): Linear(in_features=256, out_features=85, bias=True)
)



###### Declaring the hyperparameters

In [0]:
''''TODO: Try changing the hyperparameters in the network to see how it affects performance'''
batch_size = 10
seq_length = 50
n_epochs = 1  # start smaller if you are just testing initial behavior


##### Train the model and have fun with the generated texts

In [0]:
''''TODO: compare the results from the vanilla and LSTM model'''



"'TODO: compare the results from the vanilla and LSTM model"

NOTE: Training with gpu did not work, due to kernel crash. Even for small batch_size and seq_length!

Comparison with the Val Loss:

LSTM: Epoch: 1/1... Step: 7050... Loss: 1.3565... Val Loss: 1.5238

Vanilla RNN: Epoch: 1/1... Step: 6850... Loss: 1.3496... Val Loss: 1.6123

LSTM slightly better loss-wise, as well as subjective impression of the generates text. It seems to converge a little faster and deliveres readable results earlier compared to VanillaRNN.



In [0]:
#train(lstm_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

In [0]:
train(vanilla_model, text_as_int, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=50)

Epoch: 1/1... Step: 50... Loss: 3.2971... Val Loss: 3.2741
Errnieh  nesuanuse ud nesnn h eeees nie nr saaeeenedr renis srririu  u edearheaendnseira anederusr  nrersern reuhr iaiushe  eenurensenr nhi udur henineuerehedeeerereesnee hnduuhdurn u iuee ra erens  nn e edd r  nrrne sr dna e ee seeaien  useeea d ne uasheaaeeaean  n aeeeaase  idereerehserdarnnrnhds e irduh eneneananneeaiedse aeaasd  hudsdn ee rnseerarhensr ere h ernrhennn reh n sed   se eu d eandruuanrrnhn ihrsisis irde a  uien henaneeneinne nn rdinn uarrnssniane duiinsee naehd irar e  nuihnrsa enh  nu  he ee er ddn  er niausdean nau innese edeerarsni eeinuneni unar neas   n neni neui e neuuuhndsia iehh e u d d eei r nhrrn easunsei as d  e sas nherd eeunhaerrd rseannusesi  eeseese s neen sa usednerd r nuar e ed rn er  re ra a   ad n i  hnaairread eh  ei eaneeeue   eeerineahiraae er rs  r  shueseehsi ihnr s drea  suan eeeres arenr heede shs disdnu e  aed  ea  sdedr i  ide aeheernreieed nseeauss rhaune e nhssrehes easn  erneerre 